In [1201]:
%%bash
pip3 install psycopg2-binary
pip3 install networkx
pip3 install colorama
pip3 install termcolor
pip3 install py4j
pip3 install numba
pip3 install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 KB 2.4 MB/s eta 0:00:00


In [777]:
import subprocess
import psycopg2
from pathlib import Path
import json
from json import JSONEncoder
import ctdPython
from functools import partial
import sys
from py4j.java_gateway import JavaGateway
from py4j.java_collections import SetConverter, MapConverter, ListConverter
import heapq
from dataclasses import dataclass, field
from typing import Any
#from numba import jit
#from numba.experimental import jitclass
import multiprocessing
#from ctdPython.hypergraph import HyperGraph
#from ctdPython.ctdcheck import CTDCheck
#from ctdPython.ctdcheck import Block
#from ctdPython.ctdcheck import VertSet

In [778]:
import networkx as nx
import re
import pprint
import itertools
import colorama
from termcolor import colored
import functools
colorama.init()

class Edge(object):
    def __init__(self,V,name):
        assert(type(name) == str)
        assert(type(V) == set)
        self.V = V
        self.name = name

    def __repr__(self):
        return self.name + "(" + ",".join(map(str,self.V)) + ")"

class HyperGraph(object):
    def __init__(self):
        self.V = set()
        self.E = list()
        self.edge_dict = dict()

    def grid(n, m):
        h = HyperGraph()
        hc, vc = 0, 0
        for col in range(m-1):
            for row in range(n):
                vi = '{}.{}'.format(row, col)
                vright = '{}.{}'.format(row, col+1)
                horz_name = 'H{}'.format(hc)
                hc = hc+1
                h.add_edge(set([vi, vright]),
                           horz_name)
        for col in range(m):
            for row in range(n-1):
                vi = '{}.{}'.format(row, col)
                vdown = '{}.{}'.format(row+1, col)
                vert_name = 'V{}'.format(vc)
                vc = vc+1
                h.add_edge(set([vi, vdown]),
                           vert_name)
        return h

    def copy(self):
        h = HyperGraph()
        for en, e in self.edge_dict.items():
            h.add_edge(e.V, name=en)
        return h

    def join_copy(self, x, y):
        """Copy of self with vertices x and y joined"""
        if x not in self.V or y not in self.V:
            raise ValueError('Join vertices need to be in hypergraph')
        h = HyperGraph()
        for en, e in self.edge_dict.items():
            e2 = e.V.copy()
            if y in e2:
                e2.remove(y)
                e2.add(x)
            h.add_edge(e2, name=en)
        return h

    def toHyperbench(self):
        s = []
        for en, e in sorted(self.edge_dict.items()):
            s.append('{}({}),'.format(en, ','.join(e.V)))
        return '\n'.join(s)

    def vertex_induced_subg(self, U):
        """Induced by vertex set U"""
        h = HyperGraph()
        for en, e in self.edge_dict.items():
            e2 = e.V.copy()
            e2 = e2 & U
            if e2 != set():
                h.add_edge(e2, name=en)
        return h

    def bridge_subg(self, U):
        EC = [en for en, e in self.edge_dict.items() if
              (e.V & U) != set()]
        C = self.edge_subg(EC)

        # for each component C_i of rest, compute a special edge Sp_i
        for C_i in self.separate(U):
            print(C_i)
            Sp_i_parts = [(e.V - U) for e in C.E if (e.V & C_i.V) != set()]
            Sp_i = set.union(*Sp_i_parts)
            C.add_special_edge(Sp_i)
        return C

    def edge_subg(self, edge_names):
        h = HyperGraph()
        for en in edge_names:
            if en not in self.edge_dict:
                raise ValueError('Edge >{}< not present in hypergraph'.format(en))
            h.add_edge(self.edge_dict[en].copy(), en)
        return h

    def fromHyperbench(fname):
        EDGE_RE = re.compile('\s*([\w:]+)\s?\(([^\)]*)\)')
        def split_to_edge_statements(s):
            x = re.compile('\w+\s*\([^\)]+\)')
            return list(x.findall(s))

        def cleanup_lines(rl):
            a = map(str.rstrip, rl)
            b = filter(lambda x: not x.startswith('%') and len(x) > 0, a)
            return split_to_edge_statements(''.join(b))

        def line_to_edge(l):
            m = EDGE_RE.match(l)
            name = m.group(1)
            e = m.group(2).split(',')
            e = set(map(str.strip, e))
            return name, e            

        with open(fname) as f:
            raw_lines = f.readlines()
        lines = cleanup_lines(raw_lines)

        hg = HyperGraph()
        for l in lines:
            edge_name, edge = line_to_edge(l)
            hg.add_edge(edge, edge_name)
        return hg

    def add_edge(self, edge, name):
        assert(type(edge) == set)
        obj = Edge(edge,name)
        self.edge_dict[name] = obj
        self.V.update(edge)
        self.E.append(obj)

    def add_special_edge(self, sp):
        SPECIAL_NAME = 'Special'
        # find a name first
        sp_name = None
        for i in itertools.count():
            candidate = SPECIAL_NAME + str(i)
            if candidate not in self.edge_dict:
                sp_name = candidate
                break
        self.add_edge(sp, sp_name)

    def remove_edge(self, name):
        e = self.edge_dict[name]
        del self.edge_dict[name]
        self.E.remove(e)

    def primal_nx(self):
        G = nx.Graph()
        G.add_nodes_from(self.V)
        for i, e in enumerate(self.E):
            for a, b in itertools.combinations(e.V, 2):
                G.add_edge(a, b)
        return G

    def incidence_nx(self, without=[]):
        G = nx.Graph()
        G.add_nodes_from(self.V)
        G.add_nodes_from(self.edge_dict.keys())
        for n, e in self.edge_dict.items():
            if n in without:
                continue
            for v in e.V:
                G.add_edge(n, v)
        return G

    def toPACE(self, special=[]):
        buf = list()
        vertex2int = {v: str(i) for i, v in enumerate(self.V, start=1)}
        buf.append('p htd {} {}'.format(len(self.V),
                                        len(self.E)))
        for i, ei in enumerate(sorted(self.edge_dict.items()), start=1):
            en, e = ei.V
            edgestr = ' '.join(map(lambda v: vertex2int[v], e))
            line = '{} {}'.format(i, edgestr)
            buf.append(line)

        if special is None:
            special = []
        for sp in special:
            if sp is None:
                continue
            edgestr = ' '.join(map(lambda v: vertex2int[v], sp))
            buf.append('s ' + edgestr)
        return '\n'.join(buf)

    def separation_subg(self, U, sep):
        C = HyperGraph()
        cover = U | sep
        for en, e in self.edge_dict.items():
            if e.V.issubset(cover) and not e.V.issubset(sep):
                C.add_edge(e.V, en)
        return C

    def separate(self, sep, only_vertices=False):
        """Returns list of components"""
        assert(type(sep) == set)
        primal = self.primal_nx()
        primal.remove_nodes_from(sep)
        comp_vertices = nx.connected_components(primal)
        if only_vertices:
            return list(comp_vertices)
        comps = [self.separation_subg(U, sep)
                 for U in comp_vertices]
        return comps

    def toVisualSC(self):
        vertex2int = {v: str(i) for i, v in enumerate(self.V, start=1)}
        edges = map(lambda e: map(lambda v: vertex2int[v], e.V), self.E)
        buf = []
        for e in edges:
            buf.append('{'+', '.join(e) + '}')
        return ' '.join(buf)

    def fancy_repr(self, hl=[]):
        edge_style = colorama.Fore.RED + colorama.Style.NORMAL
        vertex_style = colorama.Fore.YELLOW + colorama.Style.NORMAL
        hl_style = colorama.Fore.WHITE + colorama.Back.GREEN + colorama.Style.BRIGHT
        _reset = colorama.Style.RESET_ALL

        def color_vertex(v):
            if v in hl:
                return hl_style + v + _reset
            else:
                return vertex_style + v + _reset
        s = ''
        for en, e in sorted(self.edge_dict.items()):
            s += edge_style + en + _reset + '('
            s += ','.join(map(color_vertex, e.V))
            s += ')\n'
        return s

    def __repr__(self):
        return self.fancy_repr()

In [1352]:
class VertSet(object):
    def __init__(self,vertices):
        assert(type(vertices) == set)
        self.vertices = vertices
         
    def __hash__(self):
        finalHash = 0 
        for h in self.vertices:
            finalHash = finalHash + int(h)
        return finalHash   
        
    def __repr__(self):        
        return str(sorted(list(map(lambda s: int(s), self.vertices))))

    def __eq__(self, other):
        return type(other) == VertSet and self.vertices == other.vertices

class Block(object):
    def __init__(self,head,cover,tail):
        assert(type(head) == VertSet)
        assert(type(tail) == VertSet)
        assert(len(head.vertices.intersection(tail.vertices)) == 0) # disjoint
        self.head = head
        self.cover = cover
        self.tail = tail

    def __hash__(self):
        finalHash = 0 
        for h in self.head.vertices:
            finalHash = finalHash + hash(h)
        for t in self.tail.vertices:
            finalHash = finalHash + hash(t)
        return finalHash

    def __eq__(self, other):
        return type(other) == Block and self.head == other.head and self.tail == other.tail
            
    def __repr__(self):
        return "Block("+str(self.head)+","+str(self.tail)+","+str(self.cover)+")"


    def __lt__(self,other):
        selfVert = self.head.vertices.union(self.tail.vertices) 
        otherVert = other.head.vertices.union(other.tail.vertices) 

        return selfVert.issubset(otherVert) and self.tail.vertices.issubset(other.tail.vertices)

    # connected bags filters out any bags for which the induced subgraph over E is not connected
    def connected(self,H):
        induced = H.vertex_induced_subg(self.head.vertices)
        comps = induced.separate(set())
        return len(comps) == 1 # connected if only one connected comp

    def index(self):
        cover = list(map(lambda e: e.name, self.cover))
        return ",".join(sorted(cover))

class Node:    
    def __init__(self,bag,cover,children, weight = 0):
        assert(type(bag) == VertSet)
        self.bag = bag  # set of vertices
        self.cover = cover #set of edges
        self.children = children #set of child nodes
        self.weight = weight

    def addChild(self,child):
        self.children.append(child)

    def toString(self,depth):

        tabby = "\n " + "\t" * depth
        

        childrenReps = list()
        for child in self.children:
            childrenReps.append(child.toString(depth+1))

        return "Bag: " + str(self.bag) + " Cover: " + str(self.cover) + tabby + tabby.join(childrenReps)
    
    def __repr__(self):        
        return self.toString(1)

class NodeEncoder(JSONEncoder):
    def default(self, o):
        return {'bag': list(o.bag.vertices),
                'cover': list([{'name': e.name, 'vertices': list(e.V)} for e in o.cover]),
                'children': [self.default(c) for c in o.children]}

@dataclass
class WeightedBasis:
    weight: int
    basis: Any
    def __lt__(self, other):
        return self.weight > other.weight

class CTDOpt(object):
    def __init__(self,h):
        self.H = h                   # hypergraph
        self.root_block = Block(VertSet(set()), set(), VertSet(h.V))
        self.blocks = set([self.root_block])
        self.satisfied_block = set() # indicating which blocks are satisfied
        self.head_to_blocks = dict() # mapping heads to blocks headed by them
        self.weights = dict() # maps block to weight
        self.weights[self.root_block] = sys.maxsize
        self.sj_weights = dict()
        self.children = dict()
        self.top_children = dict()
        self.top_children[self.root_block] = []
        self.new_blocks = set()
        self.head_to_cover = dict() # cache the edge covers
        self.block_to_basis = dict() # mapping a satisfied block to its basis
        self.rootHead = None # cache the root head once found

    def addBlock(self,b):
        assert(type(b) == Block)
        if b in self.blocks:
            return # don't add same block twice
        self.blocks.add(b)
        self.head_to_cover[b.head] = b.cover
        self.new_blocks.add(b)
        # print("Is the head ", b.head ," hash:",hash(b.head)  ," already in the map ", list(self.head_to_blocks.keys()))
        # print("Answer: ", b.head in list(self.head_to_blocks.keys()))
        if b.head in self.head_to_blocks:
            self.head_to_blocks[b.head].append(b)
        else:
            self.head_to_blocks[b.head] = [b]            

        self.top_children[b] = []
        if len(b.tail.vertices) == 0: 
            # print("Block ",b," added as trivially sat.")
            self.satisfied_block.add(b)  # check if trivially satisifed
            block_index = b.index()
            if block_index in self.node_weights:
                self.weights[b] = self.node_weights[block_index]
            else:
                # single edge
                self.weights[b] = 1
            self.children[b] = set()
        else:
            self.weights[b] = sys.maxsize
        # else:
        #     self.block_dict[b] = self.hasBasis(b) # basis check

    def minimize_weights(self, topn):
        # new_blocks = blocks that were updated in the last iteration -> continue until there are no more updates
        while self.new_blocks != set():
            new = set() # keep track of newly added blocks to stop when nothing new is added
            for b in self.blocks:
                if len(b.tail.vertices) == 0:
                    # skip trivial blocks
                    continue
                bases = self.determine_bases(b, self.new_blocks)
                #print("bases: " + str(bases))
                all_children = self.top_children[b]
                for basis in bases:
                    #print("basis: " + str(basis))
                    new_weight = self.basis_weight(b, basis)
                    #print("new weight: " + str(new_weight) + ", old weight: " + str(self.weights[b]))
                    #basis_sum = sum(list(map(lambda b: self.weights[b], basis)))
                    weight = self.weights[b]
                    all_children.append(WeightedBasis(new_weight, basis))
                    if new_weight < weight:
                        self.weights[b] = new_weight
                        self.children[b] = basis
                        self.block_to_basis[b] = basis
                        new.add(b)
                #print("all children: ", all_children)
                self.top_children[b] = heapq.nlargest(topn, all_children)
                #print("sorted", self.top_children[b])
            self.new_blocks = new
        if self.weights[self.root_block] == sys.maxsize:
            print("no decomposition found")
            return None
        else:
            decomps = self.construct_tds(topn)
            #decomps = [self.construct_td()]
            print("decompositions found: ")
            for decomp in decomps:
                print(decomp)
                print("weight: ", decomp.weight, "\n")
            #print("root block children", self.children[self.root_block])
            return decomps

    def construct_td(self):
        return self.to_node(self.root_block)
    
    def construct_tds(self, topn):
        return self.to_nodes(self.root_block, topn)

    def add_weights(self, node_costs):
        self.node_weights = node_costs
        
    def add_sj_weights(self, sj_weights):
        self.sj_weights = sj_weights

    def block_weight(self, block):
        cover = list(map(lambda e: e.name, block.cover))
        block_index = ",".join(sorted(cover))
        if block_index in self.node_weights:
            return self.node_weights[block_index]
        else:
            return 1
            
    def sj_weight(self, from_b, to_b):
        cover_from = [e.name for e in from_b.cover]
        idx_from = ",".join(sorted(cover_from))
        cover_to = [e.name for e in to_b.cover]
        idx_to = ",".join(sorted(cover_to))
        idx = idx_from + "-" + idx_to

        if idx in self.sj_weights:
            return self.sj_weights[idx]
        else:
            return 10000000

    def basis_weight(self, block, basis):
        basis_sum = sum(list(map(lambda b: 0 if len(b.tail.vertices) == 0 else self.weights[b], basis)))
        sj_costs = 0
        for b in basis:
            child_basis = self.children[b]
            if len(child_basis) > 0:
                child_block = next(iter(child_basis))
                sj_costs += self.sj_weight(child_block, b)
        return self.block_weight(next(iter(basis))) + sj_costs + basis_sum

    # determine the bases of a block wrt. new blocks (one of the blocks has to be from new_blocks)
    # a basis is a set of blocks
    def determine_bases(self, b, new_blocks):
        bases = []
        #print("block: " + str(b))
        for head in self.head_to_blocks:
            #print("head: " + str(head))
            allBlocks = self.head_to_blocks[head]
            #print("allblocks: " + str(allBlocks))
            #headed_blocks = [x for x in allBlocks if x < b and not (x.head == b.head and x.tail == b.tail)]
            headed_blocks = [x for x in allBlocks if x < b and not (x.head == b.head and x.tail == b.tail)]

            #print("headed blocks: " + str(headed_blocks))

            if set(headed_blocks).intersection(new_blocks) == set():
                continue

            for ob in headed_blocks:
                if self.weights[ob] == sys.maxsize:
                    continue

            # 3. condition (for each component C_i', the block (B', C_i') is satisfied
            cond3 = True
            for ob in headed_blocks:
                if not ob in self.satisfied_block:
                    cond3 = False
            if cond3 == False:
                #print("cond3 broken")
                continue #3nd Condition violated (testing first for efficiency)

            # 1. condition (the tail of the block b is a subset of the union of
            # the tails and the head
            unionTails = set()
            # union of the tails' vertices
            for ob in headed_blocks:
                for v in ob.tail.vertices:
                    unionTails.add(v)
            # add the head's vertices
            for v in head.vertices:
                unionTails.add(v)
            if not b.tail.vertices.issubset(unionTails):
                #print("cond1 broken")
                continue # 1st Condition violated

            # 2. condition (each hyperedge partially contained in the tail of b has to be contained
            # in the union of the tails and the head)
            cond2 = True
            for e in self.H.E:
                if len(e.V.intersection(b.tail.vertices)) == 0:
                    continue # find other edge
                if not e.V.issubset(unionTails):
                    cond2 = False
                    #print("cond2 broken")
                    break
            if cond2 == False:
                continue # 2nd Condition violated

            # basis found!
            basis = set()
            for ob in headed_blocks:
                basis.add(ob)
            bases.append(basis)
            #print("bases: " + str(bases))
        if bases != []:
            self.satisfied_block.add(b)
        return bases

    def hasBasis(self,b):
        basisFound = False
        basisWitness = None
        for B in self.head_to_blocks:
            allBlocks = self.head_to_blocks[B]
            blocks = [x for x in allBlocks if x < b]

            cond3 = True
            for ob in blocks:
                if not ob in self.satisfied_block:
                    cond3 = False
            if cond3 == False:
                continue #3nd Condition violated (testing first for efficiency)

            unionTails = set()
            for ob in blocks:
                for v in ob.tail.vertices:
                    unionTails.add(v)
            for v in B.vertices:
                unionTails.add(v)
            if not  b.tail.vertices.issubset(unionTails):
                continue # 1st Condition violated
            cond2 = True
            for e in self.H.E:
                if len(e.V.intersection(b.tail.vertices)) == 0:
                    continue # find other edge
                if not e.V.issubset(unionTails):
                    cond2 = False
                    break
            if cond2 == False:
                continue # 2nd Condition violated
            basisFound = True
            basisWitness = B
            # print("The basis of ", b , " is ", B)
            # print("The blocks headed by ", B)
            # for BB in blocks:
            #     print(str(BB)+"\n")

            break
        if basisFound == True:
            self.satisfied_block.add(b)
            self.block_to_basis[b] = basisWitness
            return True
        else:
            return False

    def rootHeadFound(self):
        for head in self.head_to_blocks:
            blocks = self.head_to_blocks[head]
            allSatisfied = True
            for b in blocks:
                if not b in self.satisfied_block:
                    allSatisfied = False
            if allSatisfied == True:
                # print("Root Head is ",head)
                self.rootHead = head
                return True
        return False


    def hasDecomp(self):
        while True:            
            changed = False
            for b in self.blocks:
                if b in self.satisfied_block:
                    continue # already marked as satisfied
                res = self.hasBasis(b)
                if res == True:
                    changed = True
                    #print("Found basis for the block ", b)
                if self.rootHeadFound():
                    # print("Found decomp!")
                    return True
            if changed == False:
                # print("Nothing has changed anymore, terminating")
                return False

    def to_node(self,block):
        if not(block in self.satisfied_block):
            # print(block, " is not satisfied")
            return None  # Nothing to return if block not satisfied
        if len(block.tail.vertices) == 0:
            return Node(block.head,self.head_to_cover[block.head],list()) # leaf node
        basis = self.block_to_basis[block]

        node_children = list()
        for block_child in self.children[block]:
            if len(block_child.tail.vertices) != 0:
                node_children.append(self.to_node(block_child))

        basis_head = list(basis)[0].head
        return Node(basis_head,self.head_to_cover[basis_head],node_children)

    def to_nodes(self,block,topn):
        if not(block in self.satisfied_block):
            # print(block, " is not satisfied")
            return None  # Nothing to return if block not satisfied
        if len(block.tail.vertices) == 0:
            # print(block, " is trivial")
            return Node(block.head,self.head_to_cover[block.head],list()) # leaf node

        #print("top basis", self.children[block])
        #print("weight: ", self.weights[block])
        print("top children: ", self.top_children[block])
        nodes = []
        for weighted_basis in self.top_children[block]:
            basis = weighted_basis.basis
            node_children = list()
            for block_child in weighted_basis.basis:
                if len(block_child.tail.vertices) != 0:
                    node_children.append(self.to_node(block_child))

            basis_head = list(basis)[0].head
            nodes.append(Node(basis_head,self.head_to_cover[basis_head],node_children, weighted_basis.weight))
        return nodes

    def getDecomp(self,block):
        if not(block in self.satisfied_block):
            # print(block, " is not satisfied")
            return None  # Nothing to return if block not satisfied
        if len(block.tail.vertices) == 0:
            # print(block, " is trivial")
            return Node(block.head,self.head_to_cover[block.head],list()) # leaf node
        basis = self.block_to_basis[block]
        allBlocks = self.head_to_blocks[basis]
        blocks = [x for x in allBlocks if x < block]

        # print("Child BLocks for block ", block)
        # for bs in allBlocks:
        #     print(bs)

        children = list()
        for bs in blocks: 
            children.append(self.getDecomp(bs))
        
        return Node(basis,self.head_to_cover[basis],children)


    def getDecompRoot(self):
        if self.rootHead == None:
            return None  ## can't find decomp of whole graph if no root head

        allBlocks = self.head_to_blocks[self.rootHead]
        # print("Blocks of RootHead")
        # for bs in allBlocks:
        #     print(bs)


        blocks = [x for x in allBlocks if len(x.tail.vertices) != 0]

        # print("Non-Trivial Blocks of RootHead")
        # for bs in blocks:
        #     print(bs)

        children = list()
        for bs in blocks:
            children.append(self.getDecomp(bs))

        return Node(self.rootHead,self.head_to_cover[self.rootHead],children)

In [1380]:
import itertools

def all_choose_k(S, k):
    from itertools import chain, combinations
    return chain(*(combinations(S, kp) for kp in range(1,k+1)))
#
def all_lambdas(E, k):
    yield set()
    for es in all_choose_k(E, k):
        # yield set.union(*es)
        yield es

# (over approximates) the bags produced by the LogK algorithm
def computesoftk(h, k):
    softk = list()
    lambdas = all_lambdas(h.E,k)
    for P in lambdas:
        obj1 = set()
        if len(P) == 1:
            obj1 = P[0].V
        elif len(P) > 1:
            obj1 =functools.reduce(lambda a,b: (a).union(b),map(lambda s : s.V,P))
        for C in h.separate(obj1, only_vertices=False):
            for L in lambdas:
                obj2 = set()
                if len(L) == 1:
                    obj2 = L[0].V
                elif len(L) > 1:
                    obj2 = functools.reduce(lambda a,b: (a).union(b),map(lambda s : s.V,L))
                B = set.intersection(C.V, obj2)
                if len(B) > 1 and (B, L) not in softk:
                    softk.append((B,L))
    return softk
    
# computes the blocks of a bag by computing its components w.r.t. h
def bag_to_blocks(h,pair):
    B = pair[0]
    L = pair[1]  
    blocks = list()
    for C in h.separate(B, only_vertices=True):
        blocks.append(Block(VertSet(B),L,VertSet(C)))
    blocks.append(Block(VertSet(B),L,VertSet(set())))  # adding trivial block too
    return blocks


# computes the blocks of a bag by computing its components w.r.t. h
def bag_to_blocksConnected(h,pair):
    B = pair[0]
    L = pair[1]  
    blocks = list()
    for C in h.separate(B, only_vertices=True):
        tempBlock = Block(VertSet(B),L,VertSet(C))
        if tempBlock.connected(h):
            blocks.append(tempBlock)
    tmp = Block(VertSet(B),L,VertSet(set()))
    if tmp.connected(h):
        blocks.append(tmp)  # adding trivial block too
    return blocks


# Same as  computeosftK, but returns directly the blocks
def computesoftkBlocks(h, k):
    out = list()
    listOfLists = map(partial(bag_to_blocks,h),computesoftk(h,k))
    for ll in listOfLists:
        for l in ll:
            out.append(l)
    return out


# Same as  computeosftK, but returns directly the blocks
def computesoftkBlocksConnected(h, k):
    out = list()
    listOfLists = map(partial(bag_to_blocksConnected,h),computesoftk(h,k))
    for ll in listOfLists:
        for l in ll:
            out.append(l)
    return out

def get_best_blocks(blocks, node_to_cost):
    best_weight = dict()
    best_cover = dict()

    for b in blocks:
        idx = b.index()
        weight = 1
        if idx in node_to_cost:
            weight = node_to_cost[idx]
        cover = b.cover
        if not b in best_weight:
            best_weight[b] = weight
            best_cover[b] = cover
        else:
            if weight < best_weight[b]:
                best_weight[b] = weight
                best_cover[b] = cover
    new_blocks = []
    for b, cover in best_cover.items():
        new_blocks.append(Block(b.head, cover, b.tail))
    return new_blocks

def covers_intersect(cover1, cover2):
    atts1 = set()
    for e in cover1:
        atts1.update(e.V)
    for e in cover2:
        if set.intersection(atts1, e.V) != set():
            return True
    return False

def get_connected_semijoins(covers):
    all_sjs = list(itertools.permutations(covers, 2))

    return [sj for sj in all_sjs if covers_intersect(sj[0], sj[1])]
        

In [1393]:
REWRITE_JAR = 'rewrite-assembly-0.1.0-SNAPSHOT.jar'

from timeit import default_timer as timer

class Rewriting:
    def __init__(self, original, rewritten, features, time, drop_statements):
        self.original = original
        self.rewritten = rewritten
        self.features = features
        self.time = time
        self.drop_statements = drop_statements

class QueryRewriter:
    def __init__(self, host, database, user, password, port=5432, start_process=True):
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port
        self.jdbcString = f'jdbc:postgresql://{self.host}:{self.port}/{self.database}'

        if start_process:
            self.rewrite_process = subprocess.Popen(['java', '-jar', REWRITE_JAR], stdout=subprocess.PIPE)
            # Wait for the first line which is printed after the py4j server is started)
            line = self.rewrite_process.stdout.readline()
            print(line)

        self.gateway = JavaGateway()

        self.rewriter = self.gateway.entry_point
        self.rewriter.connect(self.jdbcString, self.database, self.user, self.password)

    def rewrite(self, query, k = 2, topn = 1):
        
        self.rewriter.connect(self.jdbcString, self.database, self.user, self.password)
        self.rewriter.rewrite(query)

        output = json.loads(Path('output/output.json').read_text())
        drop_output = json.loads(Path('output/drop.json').read_text())

        result = Rewriting(query, output['rewritten_query'], output['features'], output['time'],
                           drop_output['rewritten_query'])

        hg = HyperGraph.fromHyperbench('output/hypergraph.txt')

        acyclic = output['acyclic']

        if acyclic == True:
            print("query is acyclic. done")
            for line in output["rewritten_query"]:
                print(line + ";")
            return "acyclic"
        
        print('hg: ' + str(hg))
        ctd = CTDOpt(hg)
        print('ctd: ' + str(ctd))
        blocks = computesoftkBlocks(hg,k)
        #blocks = computesoftkBlocksConnected(hg,k)

        # keep only distinct covers
        covers_dict = dict()
        for b in blocks:
            cover = list(map(lambda e: e.name, b.cover))
            index = ",".join(sorted(cover))
            # ignore single edges
            if not index in covers_dict:
                covers_dict[index] = b.cover

        # Get non-single-edge covers
        candidate_covers = [[e.name for e in cover] for cover in covers_dict.values() if len(cover) > 1]
        print("candidate covers", candidate_covers)
        node_explain_plans = json.loads(self.rewriter.determineNodeWeightsJSON(json.dumps(candidate_covers)))
        node_to_cost = dict()
        for (cover, explain) in node_explain_plans:
            index = ",".join(sorted(cover))
            plan = json.loads(explain)[0]
            cost = plan['Plan']['Total Cost']
            node_to_cost[index] = cost
        print("node costs: " + str(node_to_cost))

        semijoins = get_connected_semijoins(covers_dict.values())
        # keep only edge names (no vertices)
        semijoins = [([e.name for e in cover1], [e.name for e in cover2]) for (cover1, cover2) in semijoins]

        sj_explain_plans = json.loads(self.rewriter.determineSemijoinWeightsJSON(json.dumps(semijoins)))

        sj_to_cost = dict()
        for (c1, c2, explain) in sj_explain_plans:
            index1 = ",".join(sorted(c1))
            index2 = ",".join(sorted(c2))
            index = index1 + "-" + index2
            plan = json.loads(explain)[0]
            sj_cost = plan['Plan']['Total Cost']
            node_costs = node_to_cost.get(index1, 0) + node_to_cost.get(index2, 0)
            #print(index, sj_cost, node_costs)
            #sj_to_cost[index] = sj_cost / 100 # - node_costs
            sj_to_cost[index] = max(sj_cost - node_costs, 1)
        print("sj costs: " + str(sj_to_cost))

        #print("\n".join(map(lambda b: str(b), blocks)))
        ctd.add_weights(node_to_cost)
        ctd.add_sj_weights(sj_to_cost)
        blocks = get_best_blocks(blocks, node_to_cost)
        
        #print("\n".join(map(lambda b: str(b), blocks)))
        # The weights have to be added before the blocks because the join costs for trivially satisfied blocks
        # are set in addBlock
        for b in blocks:
            ctd.addBlock(b)
        
        res = ctd.minimize_weights(topn)
        print("Result: ",res)

        rewritings = []
        drops = []
        tds = []

        for td in res:
            output = self.rewriter.rewriteCyclicJSON(json.dumps(td, cls=NodeEncoder))
            print(output)
            output = json.loads(Path('output/output.json').read_text())
            rewritings.append(output["rewritten_query"])
            for line in output["rewritten_query"]:
                print(line + ";")
            drop_output = json.loads(Path('output/drop.json').read_text())
            drops.append(drop_output["rewritten_query"])
            tds.append(td)

        return (rewritings, drops, tds)

    def run_rewriting(self, rewriting, drop_statements):
        conn = psycopg2.connect(
            host=self.host,
            database = self.database,
            user = self.user,
            password = self.password
        )
        cur = conn.cursor()

        import time

        start_time = timer()
        
        for query in rewriting:
            print("executing query: " + query)
            cur.execute(query)
            #print(cur.fetchone())

        print(cur.fetchone())

        elapsed_time = timer() - start_time
        print("elapsed time: ", elapsed_time)

        for drop in reversed(drop_statements):
            cur.execute(drop)
        cur.close()
        conn.close()
        return elapsed_time
        
    def close(self):
        self.rewrite_process.kill()


    

#result = rewriter.rewrite(query)
#print(result)

In [1394]:
import pandas as pd

def run_top_rewritings(rewriter, query, k = 2, topn = 1):
    (rewritings, drops, tds) = rewriter.rewrite(query, k, topn)

    results = []
    for i, rewriting in enumerate(rewritings):
        drop_statements = drops[i]
        td = tds[i]
        print("running query", i)

        time = rewriter.run_rewriting(rewriting, drop_statements)
        print("cost: ", td.weight, "time: ", time)
        results.append([i, time, td.weight])
        
        df = pd.DataFrame(results, columns = ['decomp', 'runtime', 'cost'])
        df.to_csv('results.csv')
        

In [1358]:
%%time
query = """
select
	min(l_orderkey)
from
	customer,
	orders,
	lineitem,
	supplier,
	nation,
	region
where
	c_custkey = o_custkey
	and l_orderkey = o_orderkey
	and l_suppkey = s_suppkey
	and c_nationkey = s_nationkey
	and s_nationkey = n_nationkey
	and n_regionkey = r_regionkey
"""

rewriter = QueryRewriter('postgres', 'tpch', 'tpch', 'tpch')

run_top_rewritings(rewriter, query, k = 2, topn = 5)

rewriter.close()


b'Py4j server started\n'


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
ERROR StatusLogger Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


hg: E1(0,3)
E2(0,17)
E3(19,17)
E4(19,3)
E5(42,3)
E6(42)

ctd: <__main__.CTDOpt object at 0x7486fe0203a0>
candidate covers [['E2', 'E4'], ['E2', 'E3'], ['E2', 'E5'], ['E2', 'E1'], ['E2', 'E6'], ['E4', 'E3'], ['E4', 'E5'], ['E4', 'E1'], ['E4', 'E6'], ['E3', 'E5'], ['E3', 'E1'], ['E3', 'E6'], ['E5', 'E1'], ['E5', 'E6'], ['E1', 'E6']]
node costs: {'E2,E4': 18751163453.87, 'E2,E3': 3317747.2, 'E2,E5': 32302385.37, 'E1,E2': 697468.34, 'E2,E6': 32302385.37, 'E3,E4': 1976372.63, 'E4,E5': 3610.87, 'E1,E4': 67867329.04, 'E4,E6': 215840.12, 'E3,E5': 129291429.36, 'E1,E3': 1563981508859.64, 'E3,E6': 129291429.36, 'E1,E5': 56127.12, 'E5,E6': 25.98, 'E1,E6': 3239195.02}
sj costs: {'E2-E3': 3251860.58, 'E2-E1': 604671.9, 'E2-E2,E4': 952718.2, 'E2-E2,E3': 928622.0, 'E2-E2,E5': 926940.15, 'E2-E1,E2': 926900.43, 'E2-E2,E6': 926940.15, 'E2-E3,E4': 3413916.96, 'E2-E1,E4': 38208556.13, 'E2-E3,E5': 130728973.7, 'E2-E1,E3': 4263693.18, 'E2-E3,E6': 130728973.7, 'E2-E1,E5': 605617.97, 'E2-E1,E6': 2371853.15, '

In [1251]:
query = """
SELECT MIN(p.Id) FROM posts as p, postLinks as pl, postHistory as ph, votes as v, badges as b, users as u WHERE p.Id = pl.RelatedPostId AND u.Id = p.OwnerUserId AND u.Id = b.UserId AND u.Id = ph.UserId AND u.Id = v.UserId AND p.AnswerCount>=0 AND p.FavoriteCount>=0 AND pl.LinkTypeId=1 AND ph.PostHistoryTypeId=2 AND v.CreationDate>=CAST('2010-07-20 00:00:00' AS TIMESTAMP) AND u.Reputation>=1 AND u.DownVotes>=0 AND u.DownVotes<=0 AND u.UpVotes<=439 AND u.CreationDate<=CAST('2014-08-07 11:18:45' AS TIMESTAMP)
"""

query = """
SELECT p.Id FROM posts as p, postLinks as pl, postHistory as ph, votes as v, badges as b, users as u WHERE p.Id = pl.RelatedPostId AND u.Id = p.OwnerUserId AND u.Id = b.UserId AND u.Id = ph.UserId AND u.Id = v.UserId AND p.AnswerCount>=0 AND p.FavoriteCount>=0 AND pl.LinkTypeId=1 AND ph.PostHistoryTypeId=2 AND v.CreationDate>=CAST('2010-07-20 00:00:00' AS TIMESTAMP) AND u.Reputation>=1 AND u.DownVotes>=0 AND u.DownVotes<=0 AND u.UpVotes<=439 AND u.CreationDate<=CAST('2014-08-07 11:18:45' AS TIMESTAMP)
"""

rewriter = QueryRewriter('postgres', 'stats', 'stats', 'stats')

result = rewriter.rewrite(query)

rewriter.close()

b'Py4j server started\n'


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
ERROR StatusLogger Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


query is acyclic. done
CREATE OR REPLACE VIEW E1 AS SELECT posts.id AS E1_id,posts.owneruserid AS E1_owneruserid
FROM posts AS posts
WHERE posts.answercount >= 0 AND posts.favoritecount >= 0;
CREATE OR REPLACE VIEW E3 AS SELECT posthistory.userid AS E3_userid
FROM posthistory AS posthistory
WHERE CAST(posthistory.posthistorytypeid AS INTEGER) = 2;
CREATE OR REPLACE VIEW E5 AS SELECT badges.userid AS E5_userid
FROM badges AS badges;
CREATE OR REPLACE VIEW E6 AS SELECT users.id AS E6_id
FROM users AS users
WHERE users.reputation >= 1 AND users.downvotes >= 0 AND users.downvotes <= 0 AND users.upvotes <= 439 AND users.creationdate <= TIMESTAMP '2014-08-07 11:18:45';
CREATE OR REPLACE VIEW E4 AS SELECT votes.userid AS E4_userid
FROM votes AS votes
WHERE votes.creationdate >= TIMESTAMP '2010-07-20 00:00:00';
CREATE OR REPLACE VIEW E2 AS SELECT postlinks.relatedpostid AS E2_relatedpostid
FROM postlinks AS postlinks
WHERE CAST(postlinks.linktypeid AS INTEGER) = 1;
CREATE UNLOGGED TABLE E1_sta

In [1244]:
query = """
SELECT MIN(pkp1.Person1Id)
FROM City AS CityA
JOIN City AS CityB
  ON CityB.isPartOf_CountryId = CityA.isPartOf_CountryId
JOIN City AS CityC
  ON CityC.isPartOf_CountryId = CityA.isPartOf_CountryId
JOIN Person AS PersonA
  ON PersonA.isLocatedIn_CityId = CityA.CityId
JOIN Person AS PersonB
  ON PersonB.isLocatedIn_CityId = CityB.CityId
JOIN Person AS PersonC
  ON PersonC.isLocatedIn_CityId = CityC.CityId
JOIN Person_knows_Person AS pkp1
  ON pkp1.Person1Id = PersonA.PersonId
 AND pkp1.Person2Id = PersonB.PersonId
JOIN Person_knows_Person AS pkp2
  ON pkp2.Person1Id = PersonB.PersonId
 AND pkp2.Person2Id = PersonC.PersonId
JOIN Person_knows_Person AS pkp3
  ON pkp3.Person1Id = PersonC.PersonId
 AND pkp3.Person2Id = PersonA.PersonId
"""

rewriter = QueryRewriter('postgres', 'lsqb', 'lsqb', 'lsqb')

result = rewriter.rewrite(query, k = 2)
print(result)

rewriter.close()

b'Py4j server started\n'


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
ERROR StatusLogger Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


hg: E1(7,3)
E2(9,3)
E3(11,3)
E4(7,12)
E5(13,9)
E6(11,15)
E7(13,12)
E8(13,15)
E9(12,15)

ctd: <__main__.CTDOpt object at 0x748704239390>
candidate covers [['E3', 'E4'], ['E3', 'E5'], ['E3', 'E2'], ['E3', 'E7'], ['E3', 'E8'], ['E3', 'E1'], ['E3', 'E9'], ['E3', 'E6'], ['E4', 'E5'], ['E4', 'E2'], ['E4', 'E7'], ['E4', 'E8'], ['E4', 'E1'], ['E4', 'E9'], ['E4', 'E6'], ['E5', 'E2'], ['E5', 'E7'], ['E5', 'E8'], ['E5', 'E1'], ['E5', 'E9'], ['E5', 'E6'], ['E2', 'E7'], ['E2', 'E8'], ['E2', 'E1'], ['E2', 'E9'], ['E2', 'E6'], ['E7', 'E8'], ['E7', 'E1'], ['E7', 'E9'], ['E7', 'E6'], ['E8', 'E1'], ['E8', 'E9'], ['E8', 'E6'], ['E1', 'E9'], ['E1', 'E6'], ['E9', 'E6']]
node costs: {'E3,E4': 1226658.29, 'E3,E5': 1226658.29, 'E2,E3': 1391.55, 'E3,E7': 80010472.54, 'E3,E8': 80010472.54, 'E1,E3': 1391.55, 'E3,E9': 80010472.54, 'E3,E6': 2187.97, 'E4,E5': 66614974.5, 'E2,E4': 1226658.29, 'E4,E7': 140940.88, 'E4,E8': 4345125987.5, 'E1,E4': 2187.97, 'E4,E9': 140940.88, 'E4,E6': 66614974.5, 'E2,E5': 2187.97, 'E5,E

In [1397]:
%%time
query = """
select
	min(l_orderkey)
from
	customer,
	orders,
	lineitem,
	supplier,
	nation,
	region
where
	c_custkey = o_custkey
	and l_orderkey = o_orderkey
	and l_suppkey = s_suppkey
	and c_nationkey = s_nationkey
	and s_nationkey = n_nationkey
	and n_regionkey = r_regionkey
"""

#rewriter = QueryRewriter('postgres', 'tpch', 'tpch', 'tpch', start_process=False)
rewriter = QueryRewriter('postgres', 'tpch', 'tpch', 'tpch')

result = rewriter.rewrite(query, k = 6, topn = 10)
print(result)

rewriter.close()


b'Py4j server started\n'


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
ERROR StatusLogger Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


hg: E1(0,3)
E2(0,17)
E3(19,17)
E4(19,3)
E5(42,3)
E6(42)

ctd: <__main__.CTDOpt object at 0x7487047fa6e0>
candidate covers [['E2', 'E4'], ['E2', 'E3'], ['E2', 'E5'], ['E2', 'E1'], ['E2', 'E6'], ['E4', 'E3'], ['E4', 'E5'], ['E4', 'E1'], ['E4', 'E6'], ['E3', 'E5'], ['E3', 'E1'], ['E3', 'E6'], ['E5', 'E1'], ['E5', 'E6'], ['E1', 'E6'], ['E2', 'E4', 'E3'], ['E2', 'E4', 'E5'], ['E2', 'E4', 'E1'], ['E2', 'E4', 'E6'], ['E2', 'E3', 'E5'], ['E2', 'E3', 'E1'], ['E2', 'E3', 'E6'], ['E2', 'E5', 'E1'], ['E2', 'E5', 'E6'], ['E2', 'E1', 'E6'], ['E4', 'E3', 'E5'], ['E4', 'E3', 'E1'], ['E4', 'E3', 'E6'], ['E4', 'E5', 'E1'], ['E4', 'E5', 'E6'], ['E4', 'E1', 'E6'], ['E3', 'E5', 'E1'], ['E3', 'E5', 'E6'], ['E3', 'E1', 'E6'], ['E5', 'E1', 'E6'], ['E2', 'E4', 'E3', 'E5'], ['E2', 'E4', 'E3', 'E1'], ['E2', 'E4', 'E3', 'E6'], ['E2', 'E4', 'E5', 'E1'], ['E2', 'E4', 'E5', 'E6'], ['E2', 'E4', 'E1', 'E6'], ['E2', 'E3', 'E5', 'E1'], ['E2', 'E3', 'E5', 'E6'], ['E2', 'E3', 'E1', 'E6'], ['E2', 'E5', 'E1', 'E6'], ['E4', 

In [1302]:
rewriter.close()

AttributeError: 'QueryRewriter' object has no attribute 'rewrite_process'

In [1237]:
# kill rewriter process in case it was not stopped
from psutil import process_iter
from signal import SIGTERM

for proc in process_iter():
    for conns in proc.connections(kind='inet'):
        if conns.laddr.port == 25333:
            proc.send_signal(SIGTERM)

In [687]:
%%bash
pip3 install line_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.6/717.6 KB 2.5 MB/s eta 0:00:00


In [1156]:
%load_ext line_profiler

query = """
SELECT MIN(pkp1.Person1Id)
FROM City AS CityA
JOIN City AS CityB
  ON CityB.isPartOf_CountryId = CityA.isPartOf_CountryId
JOIN City AS CityC
  ON CityC.isPartOf_CountryId = CityA.isPartOf_CountryId
JOIN Person AS PersonA
  ON PersonA.isLocatedIn_CityId = CityA.CityId
JOIN Person AS PersonB
  ON PersonB.isLocatedIn_CityId = CityB.CityId
JOIN Person AS PersonC
  ON PersonC.isLocatedIn_CityId = CityC.CityId
JOIN Person_knows_Person AS pkp1
  ON pkp1.Person1Id = PersonA.PersonId
 AND pkp1.Person2Id = PersonB.PersonId
JOIN Person_knows_Person AS pkp2
  ON pkp2.Person1Id = PersonB.PersonId
 AND pkp2.Person2Id = PersonC.PersonId
JOIN Person_knows_Person AS pkp3
  ON pkp3.Person1Id = PersonC.PersonId
 AND pkp3.Person2Id = PersonA.PersonId
"""



def profile():
    rewriter = QueryRewriter('postgres', 'lsqb', 'lsqb', 'lsqb')

    result = rewriter.rewrite(query, k = 9, topn = 1)
    print(result)

    rewriter.close()

%lprun -f QueryRewriter.rewrite profile()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
b'Py4j server started\n'


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
ERROR StatusLogger Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


hg: E1(7,3)
E2(9,3)
E3(11,3)
E4(7,12)
E5(13,9)
E6(11,15)
E7(13,12)
E8(13,15)
E9(12,15)

ctd: <__main__.CTDOpt object at 0x74870758d540>
candidate covers [['E3', 'E4'], ['E3', 'E5'], ['E3', 'E2'], ['E3', 'E7'], ['E3', 'E8'], ['E3', 'E1'], ['E3', 'E9'], ['E3', 'E6'], ['E4', 'E5'], ['E4', 'E2'], ['E4', 'E7'], ['E4', 'E8'], ['E4', 'E1'], ['E4', 'E9'], ['E4', 'E6'], ['E5', 'E2'], ['E5', 'E7'], ['E5', 'E8'], ['E5', 'E1'], ['E5', 'E9'], ['E5', 'E6'], ['E2', 'E7'], ['E2', 'E8'], ['E2', 'E1'], ['E2', 'E9'], ['E2', 'E6'], ['E7', 'E8'], ['E7', 'E1'], ['E7', 'E9'], ['E7', 'E6'], ['E8', 'E1'], ['E8', 'E9'], ['E8', 'E6'], ['E1', 'E9'], ['E1', 'E6'], ['E9', 'E6'], ['E3', 'E4', 'E5'], ['E3', 'E4', 'E2'], ['E3', 'E4', 'E7'], ['E3', 'E4', 'E8'], ['E3', 'E4', 'E1'], ['E3', 'E4', 'E9'], ['E3', 'E4', 'E6'], ['E3', 'E5', 'E2'], ['E3', 'E5', 'E7'], ['E3', 'E5', 'E8'], ['E3', 'E5', 'E1'], ['E3', 'E5', 'E9'], ['E3', 'E5', 'E6'], ['E3', 'E2', 'E7'], ['E3', 'E2', 'E8'], ['E3', 'E2', 'E1'], ['E3', 'E2', 'E9'], ['

Timer unit: 1e-09 s

Total time: 3.3331 s
File: /tmp/ipykernel_52847/4247542788.py
Function: rewrite at line 31

Line #      Hits         Time  Per Hit   % Time  Line Contents
    31                                               def rewrite(self, query, k = 2, topn = 1):
    32                                                   
    33         1   22596533.0    2e+07      0.7          self.rewriter.connect(self.jdbcString, self.database, self.user, self.password)
    34         1 1111331857.0    1e+09     33.3          self.rewriter.rewrite(query)
    35                                           
    36         1     318388.0 318388.0      0.0          output = json.loads(Path('output/output.json').read_text())
    37         1     152106.0 152106.0      0.0          drop_output = json.loads(Path('output/drop.json').read_text())
    38                                           
    39         2      13355.0   6677.5      0.0          result = Rewriting(query, output['rewritten_query'], 

In [1257]:
%%bash

#apt install jq

cat output/output.json | jq -r '.rewritten_query | join("; \n")'


CREATE OR REPLACE VIEW E1 AS SELECT posts.id AS E1_id,posts.owneruserid AS E1_owneruserid
FROM posts AS posts
WHERE posts.answercount >= 0 AND posts.favoritecount >= 0; 
CREATE OR REPLACE VIEW E3 AS SELECT posthistory.userid AS E3_userid
FROM posthistory AS posthistory
WHERE CAST(posthistory.posthistorytypeid AS INTEGER) = 2; 
CREATE OR REPLACE VIEW E5 AS SELECT badges.userid AS E5_userid
FROM badges AS badges; 
CREATE OR REPLACE VIEW E6 AS SELECT users.id AS E6_id
FROM users AS users
WHERE users.reputation >= 1 AND users.downvotes >= 0 AND users.downvotes <= 0 AND users.upvotes <= 439 AND users.creationdate <= TIMESTAMP '2014-08-07 11:18:45'; 
CREATE OR REPLACE VIEW E4 AS SELECT votes.userid AS E4_userid
FROM votes AS votes
WHERE votes.creationdate >= TIMESTAMP '2010-07-20 00:00:00'; 
CREATE OR REPLACE VIEW E2 AS SELECT postlinks.relatedpostid AS E2_relatedpostid
FROM postlinks AS postlinks
WHERE CAST(postlinks.linktypeid AS INTEGER) = 1; 
CREATE UNLOGGED TABLE E1_stage1_1 AS SELECT *